In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras import layers, models
from sklearn.metrics import classification_report 

2023-09-25 18:47:33.018025: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def get_data():
    train_data = "Data/train"
    val_data = "Data/val"
    return train_data, val_data

In [3]:
image_size = 299
batch_size = 16
def get_preprocessed_data(train_data, val_data):
    train_datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale = 1./255,
        zoom_range= 0.2,
        horizontal_flip = True
    )
    test_val_datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
    )
    train_generator = train_datagen.flow_from_directory(
        directory = train_data,
        target_size=(image_size, image_size),
        class_mode="categorical",
        batch_size=batch_size,
        shuffle=True
    )
    val_generator = test_val_datagen.flow_from_directory(
        directory = val_data,
        target_size=(image_size, image_size),
        class_mode="categorical",
        batch_size=batch_size,
        shuffle=False
    )
    
    return train_generator, val_generator 

In [4]:
train_data, val_data = get_data()
train_ds, val_ds = get_preprocessed_data(train_data, val_data)

Found 2116 images belonging to 4 classes.
Found 601 images belonging to 4 classes.


In [8]:
net = keras.applications.inception_v3.InceptionV3(
        weights="imagenet",
        include_top=False,
        input_shape=(image_size, image_size, 3)
    )
net.summary()   
for layer in net.layers[:-2]:
    layer.trainable = False

x = layers.GlobalAveragePooling2D()(net.output)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
prediction = layers.Dense(4, activation="softmax")(x)
model = models.Model(inputs = net.input, outputs = prediction)
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 149, 149, 32)         864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 149, 149, 32)         96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activation)  (None, 149, 149, 32)         0         ['batch_normalizati

In [9]:
model.compile(
        loss = "categorical_crossentropy",
        optimizer = keras.optimizers.Adam(learning_rate=0.001),
        metrics = ['acc']
)

In [10]:
history = model.fit(
        train_ds,
        epochs = 10,
        validation_data=val_ds
)

Epoch 1/10
133/133 [==============================] - 380s 3s/step - loss: 0.5832 - acc: 0.7741 - val_loss: 0.6954 - val_acc: 0.6423
Epoch 2/10
133/133 [==============================] - 377s 3s/step - loss: 0.3860 - acc: 0.8544 - val_loss: 0.6476 - val_acc: 0.7121
Epoch 3/10
133/133 [==============================] - 376s 3s/step - loss: 0.3307 - acc: 0.8762 - val_loss: 0.4441 - val_acc: 0.8203
Epoch 4/10
133/133 [==============================] - 373s 3s/step - loss: 0.3149 - acc: 0.8814 - val_loss: 0.4437 - val_acc: 0.8120
Epoch 5/10
133/133 [==============================] - 371s 3s/step - loss: 0.2745 - acc: 0.9036 - val_loss: 0.4773 - val_acc: 0.8087
Epoch 6/10
133/133 [==============================] - 372s 3s/step - loss: 0.2785 - acc: 0.8970 - val_loss: 0.4343 - val_acc: 0.8136
Epoch 7/10
133/133 [==============================] - 373s 3s/step - loss: 0.2587 - acc: 0.9017 - val_loss: 0.4646 - val_acc: 0.8153
Epoch 8/10
133/133 [==============================] - 373s 3s/step - 

In [11]:
scores = model.evaluate(val_ds, verbose=1)
print(f"Loss: {scores[0]}, Accuracy : {scores[1]}")

38/38 [==============================] - 71s 2s/step - loss: 0.3747 - acc: 0.8602
Loss: 0.3747063875198364, Accuracy : 0.860232949256897


In [12]:
test_pred = model.predict(val_ds, verbose = 1)
test_labels = np.argmax(test_pred, axis=1)

38/38 [==============================] - 73s 2s/step


In [14]:
class_labels = val_ds.class_indices
class_labels = {v:k for k,v in class_labels.items()}
classes = list(class_labels.values())
print(classes)
print('Classification Report')
print(classification_report(val_ds.classes, test_labels, target_names=classes))

['COVID19', 'NORMAL', 'PNEUMONIA', 'TURBERCULOSIS']
Classification Report
               precision    recall  f1-score   support

      COVID19       0.86      0.92      0.89       106
       NORMAL       0.86      0.72      0.78       164
    PNEUMONIA       0.79      0.87      0.83       180
TURBERCULOSIS       0.95      0.95      0.95       151

     accuracy                           0.86       601
    macro avg       0.87      0.87      0.86       601
 weighted avg       0.86      0.86      0.86       601



In [15]:
model.save("model/XRayInceptionV3.h5")

/Users/dipit/anaconda3/envs/tensorflow/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
